In [6]:
import pandas as pd
import numpy as np

In [13]:
# from scipy.stats import shapiro
from scipy.stats import anderson
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from pyclustertend import hopkins

In [14]:
from sklearn.ensemble import IsolationForest
from pyod.models.mcd import MCD

In [12]:
data = np.random.normal(size=(10000,2))
len(data)

10000

In [123]:
# anderson(data).statistic < anderson(data).critical_values[2]

### Experimets regarding Algorithm

In [124]:
sc = StandardScaler()
x_sc = sc.fit_transform(data.reshape(-1,1))
count = 0
for i in x_sc:
    if i > 3 or i < -3:
        count += 1
count
x_sc.flatten().ndim

1

In [125]:
# mean = data.mean()
# std = data.std()
# x_sc = np.array([(i-mean)/std for i in data])
# count = 0
# for i in x_sc:
#     if i > 3 or i < -3:
#         count += 1
# count

filtered_data
# count = 0
# for i in filtered_data:
#     if i > 3 or i < -3:
#         count += 1
# count

len(data) - len(filtered_data)

32

In [132]:
filtered_data = x_sc[(-3 <= x_sc) & (x_sc <= 3)]
outliers = x_sc[(x_sc > 3) | (x_sc < -3)]
# outliers
df = pd.DataFrame(data)
# data - filtered_data

In [127]:
model = KMeans(n_clusters=2,random_state=0,n_init='auto')
model.fit(filtered_data.reshape(-1,1))
labels = model.labels_
print(len(labels))
count = 0
# #Labelling clusters and outliers
# df['label'] = -1
# # df.loc[df.index.isin([i[1] for i in enumerate(filtered_data)]), 'label'] = labels 

# # unique_indices = 
# df.loc[np.unique(np.where((-3 <= x_sc) & (x_sc <= 3))[0]), 'label'] = labels        
# df['label'].value_counts()
all_labels = -1 * np.ones_like(x_sc.flatten())  # Initialize all as outliers
# filtered_indices = np.unique(np.where((-3 <= x_sc) & (x_sc <= 3))[0])
all_labels[np.unique(np.where((-3 <= x_sc) & (x_sc <= 3))[0])] = labels
for i in all_labels:
    if i == -1:
        count +=1
count

9976


24

In [20]:
data = np.random.normal(size=(10000,2))
df = pd.DataFrame(data)
model = IsolationForest(contamination=0.0026)
df['anomaly'] = model.fit_predict(df)

In [21]:
np.array(df.anomaly)

array([1, 1, 1, ..., 1, 1, 1])

In [22]:
df.anomaly.value_counts()

anomaly
 1    9974
-1      26
Name: count, dtype: int64

### Algorithm

In [38]:
def outlier_KMeans(data,contamination=0.05,clustering=True):
    data = np.array(data)
    data_dimension = data.ndim
    if data_dimension == 1:
        # Single Dimensional Data
        if anderson(data).statistic < anderson(data).critical_values[2]:
            # data_distribution = 'normal'
            sc = StandardScaler()
            x_sc = sc.fit_transform(data.reshape(-1,1))
            
            if clustering == False:
                outliers = x_sc[(x_sc > 3) | (x_sc < -3)]
                labels = np.ones(len(data))
                outlier_mask = np.isin(data, outliers)
                labels[outlier_mask] = -1
                return labels
            else:
                filtered_data = x_sc[(-3 <= x_sc) & (x_sc <= 3)]
                hopkins_stats = hopkins(filtered_data,len(filtered_data)//10)
                if hopkins_stats < 0.5:
                    return ('Data can not be clustered')
                else:
                    k_values = range(2, 11)
                    models = [KMeans(n_clusters=k, random_state=42,n_init='auto').fit(filtered_data.reshape(-1,1)) for k in k_values]
                    labels = [model.labels_ for model in models]
                    sil_score = [silhouette_score(data,label) for label in labels]
                    optimal_k = k_values[np.argmax(sil_score)]
                    
                    model = KMeans(n_clusters=optimal_k,random_state=0,n_init='auto')
                    model.fit(filtered_data.reshape(-1,1))
                    labels = model.labels_
                    cluster_label = -1 * np.ones_like(x_sc.flatten())
                    cluster_label[np.unique(np.where((-3 <= x_sc) & (x_sc <= 3))[0])] = labels
                    return cluster_label
        
        else:
            # data_distribution = 'non_normal'
            Q1 = np.quantile(data,0.25)
            Q3 = np.quantile(data,0.75)
            IQR = Q3-Q1
            lw = Q1 - 1.5*IQR
            rw = Q3 + 1.5*IQR
            
            if clustering == False:
                outliers = data[(data > rw) | (data < lw)]
                labels = np.ones(len(data))
                outlier_mask = np.isin(data, outliers)
                labels[outlier_mask] = -1
                return labels
            else:
                filtered_data = data[(lw <= data) & (data <= rw)]
                hopkins_stats = hopkins(filtered_data,len(filtered_data)//10)
                if hopkins_stats < 0.5:
                    return ('Data can not be clustered')
                else:
                    k_values = range(2, 11)
                    models = [KMeans(n_clusters=k, random_state=42,n_init='auto').fit(filtered_data.reshape(-1,1)) for k in k_values]
                    labels = [model.labels_ for model in models]
                    sil_score = [silhouette_score(data,label) for label in labels]
                    optimal_k = k_values[np.argmax(sil_score)]
                    
                    model = KMeans(n_clusters=optimal_k,random_state=0,n_init='auto')
                    model.fit(filtered_data.reshape(-1,1))
                    labels = model.labels_
                    cluster_label = -1 * np.ones_like(x_sc.flatten())
                    cluster_label[np.unique(np.where((-3 <= x_sc) & (x_sc <= 3))[0])] = labels
                    return cluster_label
    else:
        # Multi Dimensional Data
        if shapiro(data)[1] < 0.05:
            #data_distribution = 'non_normal'
            df = pd.DataFrame(data)
            model = IsolationForest(contamination=contamination)
            df['anomaly'] = model.fit_predict(df)
            
            if clustering == False:
                return np.array(df['anomaly'])
            else:
                fil = df['anomaly'] == -1
                filtered_data = df.loc[fil].iloc[:,:-1].values
                hopkins_stats = hopkins(filtered_data,len(filtered_data)//10)
                if hopkins_stats < 0.5:
                    return ('Data can not be clustered')
                else:
                    k_values = range(2, 11)
                    models = [KMeans(n_clusters=k, random_state=42,n_init='auto').fit(filtered_data) for k in k_values]
                    labels = [model.labels_ for model in models]
                    sil_score = [silhouette_score(data,label) for label in labels]
                    optimal_k = k_values[np.argmax(sil_score)]
                    
                    model = KMeans(n_clusters=optimal_k,random_state=42,n_init='auto')
                    model.fit(filtered_data)
                    labels = model.labels_
                    df.loc[df['anomaly'] == -1,'label'] = -1
                    df.loc[df['anomaly'] == 1,'label'] = labels
                    return np.array(df['label'])
                
        else:
            # data_distribution = 'normal'
            df = pd.DataFrame(data)
            model = MCD(contamination=contamination)
            model.fit(data)
            df['anomaly'] = model.predict(data)
            df['anomaly'] = df['anomaly'].replace({0: 1, 1: -1})

            if clustering == False:
                return np.array(df['anomaly'])
            else:
                fil = df['anomaly'] == -1
                filtered_data = df.loc[fil].iloc[:,:-1].values
                hopkins_stats = hopkins(filtered_data,len(filtered_data)//10)
                if hopkins_stats < 0.5:
                    return ('Data can not be clustered')
                else:
                    k_values = range(2, 11)
                    models = [KMeans(n_clusters=k, random_state=42,n_init='auto').fit(filtered_data) for k in k_values]
                    labels = [model.labels_ for model in models]
                    sil_score = [silhouette_score(data,label) for label in labels]
                    optimal_k = k_values[np.argmax(sil_score)]
                    
                    model = KMeans(n_clusters=optimal_k,random_state=42,n_init='auto')
                    model.fit(filtered_data)
                    labels = model.labels_
                    df.loc[df['anomaly'] == -1,'label'] = -1
                    df.loc[df['anomaly'] == 1,'label'] = labels
                    return np.array(df['label'])

In [129]:
model = outlier_KMeans(data,k=2)
# len(data)

In [130]:
len(model[1])

24

In [39]:
data = np.random.normal(25,5,size=10000)

In [41]:
sc = StandardScaler()
x_sc = sc.fit_transform(data.reshape(-1,1))

In [43]:
count = 0
for i in data:
    if i < -3 or i > 3:
        count += 1
count

10000

In [42]:
count = 0
for i in x_sc:
    if i < -3 or i > 3:
        count += 1
count

30